# CineSuggest

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import re
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /Users/robert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
df = pd.read_csv('imdb.csv')

In [32]:
df.head(20)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
5,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"
6,https://m.media-amazon.com/images/M/MV5BNGNhMD...,Pulp Fiction,1994,A,154 min,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",94.0,Quentin Tarantino,John Travolta,Uma Thurman,Samuel L. Jackson,Bruce Willis,1826188,"107,928,762"
7,https://m.media-amazon.com/images/M/MV5BNDE4OT...,Schindler's List,1993,A,195 min,"Biography, Drama, History",8.9,"In German-occupied Poland during World War II,...",94.0,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,"96,898,818"
8,https://m.media-amazon.com/images/M/MV5BMjAxMz...,Inception,2010,UA,148 min,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,2067042,"292,576,195"
9,https://m.media-amazon.com/images/M/MV5BMmEzNT...,Fight Club,1999,A,139 min,Drama,8.8,An insomniac office worker and a devil-may-car...,66.0,David Fincher,Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier,1854740,"37,030,102"


In [4]:
# df['Series_Title'] = df['Series_Title'].apply(lambda i: i.split(','))

In [5]:
# df['Series_Title'][1]

In [6]:
df['Overview'] = df['Overview'].str.lower()
df['Overview'] = df['Overview'].apply(lambda i: re.sub('[^a-zA-Z]', ' ', i))
df['Overview'] = df['Overview'].apply(lambda i: re.sub('\s+', ' ', i))

In [7]:
# df['Overview']

In [8]:
df['Overview'] = df['Overview'].apply(lambda i: nltk.word_tokenize(i))
# df['Overview']

In [9]:
word_rem = nltk.corpus.stopwords.words('english')
temp_list = []
for sent in df['Overview']:
    temp = []
    for word in sent:
        if word not in word_rem and len(word) >= 3:
            temp.append(word)
    temp_list.append(temp)
# temp_list

In [10]:
df['Overview'] = temp_list
# df['Overview']

In [11]:
df['Genre'] = df['Genre'].apply(lambda i: i.split(','))
# df['Genre'][0]

In [12]:
df['Actors'] = df['Star1'] +','+ df['Star2'] +','+ df['Star3'] +','+ df['Star4']
df['Actors'] = df['Actors'].apply(lambda i: i.split(','))

In [13]:
# df['Actors'][85]

In [14]:
df['Director'] = df['Director'].apply(lambda i: i.split(','))
# df['Director'][0]

In [15]:
def lowlow(lst):
    temp_lst = []
    for i in lst:
        temp_lst.append(i.lower().replace(' ', ''))
    return temp_lst

In [16]:
df['Genre'] = [lowlow(x) for x in df['Genre']]
df['Actors'] = [lowlow(x) for x in df['Actors']]
df['Director'] = [lowlow(x) for x in df['Director']]

In [17]:
# df['Director'][0]

In [18]:
# df['Genre'][0]

In [19]:
# df['Actors'][0]

In [20]:
col_list = ['Overview', 'Genre', 'Director', 'Actors']
new_list = []
for i in range(len(df)):
    words = ''
    for col in col_list:
        words += ' '.join(df[col][i]) + ' '
    new_list.append(words)

In [21]:
# new_list

In [22]:
df['Overview'] = new_list
df = df[['Series_Title', 'Overview']]
# df.head()

In [23]:
# computes the term frequency (TF) for each term in each document, which is the number of 
# times a term appears in a document

tfi_vect = TfidfVectorizer()

# fits the vectorizer to the training data
feats = tfi_vect.fit_transform(df['Overview'])

In [24]:
# looks for similiarites in words in Overview rows
cos_sim = cosine_similarity(feats, feats)
# print(cos_sim)

In [25]:
index = pd.Series(df['Series_Title'])
# index.head()

In [26]:
def suggest(title):
    movies = []
    idx = index[index == title].index[0]
    # print(idx)
    sims = pd.Series(cos_sim[idx]).sort_values(ascending=False)
    top3 = list(sims.iloc[1:4].index)
    # print(top3)
    
    for i in top3:
        movies.append(df['Series_Title'][i])
    return movies

In [27]:
suggest('12 Angry Men')

['On the Waterfront', 'The Exorcist', 'Road to Perdition']

In [28]:
user_list = ['Blade Runner', 'Donnie Darko', 'Dazed and Confused', 'Alien', 'Prometheus']

In [44]:
suggest('American Beauty')

['Big Fish', 'Road to Perdition', 'Manhattan']